
# Watershed Dynamics Model 

<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:300px;padding:20px">   

Explore recharge and landsliding in the North Cascades National Park from a HydroShare Observatory. <br />
<br />
This Jupyter Notebook contains selected model components <br /> 
of the Watershed Dynamics model identified with a bullet  <br /> 
<br /> 
* Download daily 1/16 degree gridded meteorology data, <br /> 
* Download daily 1/16 degree VIC fluxes.<br /> 
* Save data to a new HydroShare resource, <br /> 

##  HydroShare Setup and Preparation

To run this notebook, we must import several libaries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation. 

In [ ]:
import os
from utilities import hydroshare #, observatory_localimport_timeseries
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import pandas as pd
import shutil

# set variables for interacting with HydroShare from this notebook
hs=hydroshare.hydroshare()

In [ ]:
#This is temporary until the .py is uploaded to HydroShare and Landlab utilities
# create source paths for the observatory scripts
Dest_dir = '/'.join(['/home/jovyan/work/notebooks','data',os.environ['HS_RES_ID'],os.environ['HS_RES_ID'],'data','contents'])
obs_sx_srcs = ['/'.join([Dest_dir, paths]) for paths in os.listdir(Dest_dir) if paths.startswith('observatory_') and paths.endswith('.py')]

# create the destination directory
obs_sx_dest = '/'.join(['/home/jovyan/work/notebooks','utilities'])

# copy the observatory scripts into the utilities folder
for eachsrc in obs_sx_srcs:
    shutil.copy(eachsrc, obs_sx_dest)
    print(eachsrc)

In [ ]:
from utilities import observatory_gridded_meteorology as ogm

Next we need to establish a secure connection with HydroShare. This is done by simply instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving your work back to HydroShare. 

In [ ]:
# Create object to map the home directory
homedir = Dest_dir + '/'
print homedir

# if the working directory is not starting at homedir, set it to homedir
if os.getcwd() != homedir:
    os.chdir(homedir)

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## 2. Get list of gridded climate points for the watershed
Retrieve a list of grid points and configuration file from the downloaded HydroShare resource

In [ ]:
# identify the file containing lat/long points
mappingfile = os.path.join(homedir,'NOCA_280_Centroid_GriddedMet.csv')
print mappingfile

 ## 3. Download gridded climate data 


### Get Daily Meteorologic Data <br/> from Livneh et al. 2013 (1915-2011; CONUS extent) 
Please cite 2013 data with: <br/>
Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.


## User Note:

Run the next two code blocks if your data is within the North America area and you want daily data 1950-2013. View the data extent from this HydroShare resource: Livneh, B. (2017). Gridded climatology locations (1/16th degree): North American extent, HydroShare,  http://www.hydroshare.org/resource/ef2d82bf960144b4bfb1bae6242bcc7f

### Get VIC fluxes (1915-2011) from Livneh et al. 2013

Please cite: <br/>
Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.

In [ ]:
# read in the longitude and latitude points from the reference mapping file
Daily_VIC_1915_2011 = ogm.getClimateData_DailyVIC_livneh2013(homedir, mappingfile)

## 4. Save the results back into HydroShare
<a name="creation"></a>

Using the `hs_utils` library, the results of the Geoprocessing steps above can be saved back into HydroShare.  First, define all of the required metadata for resource creation, i.e. *title*, *abstract*, *keywords*, *content files*.  In addition, we must define the type of resource that will be created, in this case *genericresource*.  

***Note:*** Make sure you save the notebook at this point, so that all notebook changes will be saved into the new HydroShare resource.


***Option A*** : define the resource from which this "NEW" content has been derived.  This is one method for tracking resource provenance.

In [ ]:
if 'Daily_VIC_1915_2011' not in locals():
    Daily_VIC_1915_2011 = ''.join([homedir, 'livneh2013/', 'Daily_VIC_1915_2011/'])
print Daily_VIC_1915_2011
print homedir

In [ ]:
# create a list of files with their paths to be added to the HydroShare resource.
files = ogm.compileContentfiles(Daily_VIC_1915_2011)
print files[0]  #print example files

In [ ]:
# for each file downloaded onto the server folder, move to a new HydroShare Generic Resource
title = 'Climate Data Download from NOCA Observatory' # title for the new resource
abstract = 'This a download of climate data and vizualization processing results from the Daily_VIC_1915_2011 (Livneh et al. 2013); Livneh B., E.A. Rosenberg, C. Lin, B. Nijssen, V. Mishra, K.M. Andreadis, E.P. Maurer, and D.P. Lettenmaier, 2013: A Long-Term Hydrologically Based Dataset of Land Surface Fluxes and States for the Conterminous United States: Update and Extensions, Journal of Climate, 26, 9384–9392.' # abstract for the new resource
keywords = ['Livneh', 'climate', 'VIC'] # keywords for the new resource
rtype = 'genericresource'          # Hydroshare resource type

# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          derivedFromId=os.environ['HS_RES_ID'],
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)